In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler

import warnings
warnings.filterwarnings('ignore')


In [ ]:
df = pd.read_csv('Rainfall.csv')
df.head()

In [ ]:
df.shape

(366, 12)

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
df.rename(str.strip,
axis='columns', 
inplace=True)

df.columns


In [ ]:
for col in df.columns:

# Checking if the column contains
# any null values
if df[col].isnull().sum() > 0:
val = df[col].mean()
df[col] = df[col].fillna(val)

df.isnull().sum().sum()


0

In [ ]:
plt.pie(df['rainfall'].value_counts().values,
labels = df['rainfall'].value_counts().index,
autopct='%1.1f%%')
plt.show()


In [ ]:
df.groupby('rainfall').mean()

In [ ]:
features = list(df.select_dtypes(include = np.number).columns)
features.remove('day')
print(features)


['pressure', 'maxtemp', 'temperature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']

In [ ]:
plt.subplots(figsize=(15,8))

for i, col in enumerate(features):
plt.subplot(3,4, i + 1)
sb.distplot(df[col])
plt.tight_layout()
plt.show()


In [ ]:
plt.subplots(figsize=(15,8))

for i, col in enumerate(features):
plt.subplot(3,4, i + 1)
sb.boxplot(df[col])
plt.tight_layout()
plt.show()


In [ ]:
df.replace({'yes':1, 'no':0}, inplace=True)

In [ ]:
plt.figure(figsize=(10,10))
sb.heatmap(df.corr() > 0.8,
annot=True,
cbar=False)
plt.show()


In [ ]:
df.drop(['maxtemp', 'mintemp'], axis=1, inplace=True)

In [ ]:
features = df.drop(['day', 'rainfall'], axis=1)
target = df.rainfall

In [ ]:
X_train, X_val, \
Y_train, Y_val = train_test_split(features,
target,
test_size=0.2,
stratify=target,
random_state=2)

# As the data was highly imbalanced we will
# balance it by adding repetitive rows of minority class.
ros = RandomOverSampler(sampling_strategy='minority',
random_state=22)
X, Y = ros.fit_resample(X_train, Y_train)


In [ ]:
# Normalizing the features for stable and fast training.
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_val = scaler.transform(X_val)


In [ ]:
models = [LogisticRegression(), XGBClassifier(), SVC(kernel='rbf', probability=True)]

for i in range(3):
models[i].fit(X, Y)

print(f'{models[i]} : ')

train_preds = models[i].predict_proba(X) 
print('Training Accuracy : ', metrics.roc_auc_score(Y, train_preds[:,1]))

val_preds = models[i].predict_proba(X_val) 
print('Validation Accuracy : ', metrics.roc_auc_score(Y_val, val_preds[:,1]))
print()


In [ ]:
metrics.plot_confusion_matrix(models[2], X_val, Y_val)
plt.show()

In [ ]:
print(metrics.classification_report(Y_val,
models[2].predict(X_val)))
